# 02. Registration to MNI Space

This notebook handles the registration of the subject's 7T T1w image to the MNI standard space template. We then apply this transform to the R2* map.

In [ ]:
import sys
sys.path.append('../')
import os
from src.io import find_bids_files
from src.preprocessing import register_to_mni, apply_transforms
from src.visualization import plot_slice_visualization
import nibabel as nib

## 1. Define Paths

We need an MNI template. `antspyx` or `nilearn` can provide one, or we can download a specific high-res one.

In [ ]:
# Example using MNI152 from nilearn/antspy logic or a local file
# For this pipeline, we assume you have an MNI template in 'atlases/MNI152_T1_1mm.nii.gz' or similar.
# If not, we can fetch one using nilearn.

from nilearn.datasets import fetch_icbm152_2009
mni = fetch_icbm152_2009()
mni_template_path = mni['t1']
print(f"Using MNI template: {mni_template_path}")

## 2. Perform Registration

Register T1w -> MNI.

In [ ]:
DATA_DIR = '../data'
OUTPUT_DIR = '../outputs/results'
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

subjects = [d for d in os.listdir(DATA_DIR) if d.startswith('sub-')}

if subjects:
    sub_id = subjects[0]
    t1w_files = find_bids_files(DATA_DIR, sub_id, 'T1w.nii.gz')
    
    if t1w_files:
        t1w_path = t1w_files[0]
        output_prefix = os.path.join(OUTPUT_DIR, f"{sub_id}_to_MNI")
        
        print("Starting registration... this may take a while.")
        reg_results = register_to_mni(mni_template_path, t1w_path, output_prefix)
        
        print("Registration complete.")
        # Visualize
        plot_slice_visualization(reg_results['warped_image_path'], title="Warped T1w")

## 3. Apply Transform to R2*

We use the transforms calculated above to warp the R2* map to MNI space.

In [ ]:
    r2s_files = find_bids_files(DATA_DIR, sub_id, 'R2star.nii.gz')
    if r2s_files:
        r2s_path = r2s_files[0]
        r2s_output_path = os.path.join(OUTPUT_DIR, f"{sub_id}_R2star_MNI.nii.gz")
        
        # Transform list from registration (fwdtransforms)
        transforms = reg_results['fwdtransforms']
        
        print("Applying transforms to R2*...")
        warped_r2s = apply_transforms(mni_template_path, r2s_path, transforms, r2s_output_path)
        
        plot_slice_visualization(warped_r2s, title="Warped R2*", cmap='hot')